In [ ]:
import torch
from torch.utils.data import DataLoader
import pickle
import pandas as pd

from src.custom_inference_dataset import CustomInferenceDataset
from src.handler import Handler

In [2]:
with open('./data/x_num_categories_list.pkl', 'rb') as f:
	x_num_categories_list = pickle.load(f)

with open('./data/y_num_categories_list.pkl', 'rb') as f:
	y_num_categories_list = pickle.load(f)

with open('./data/label_encoders.pkl', 'rb') as f:
	label_encoders = pickle.load(f)

with open('./data/onehot_encoders.pkl', 'rb') as f:
	one_hot_encoders = pickle.load(f)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Handler(x_num_categories_list=x_num_categories_list, y_num_categories_list=y_num_categories_list)

model.load_state_dict(torch.load('./models/best_model.pth', map_location=device))

model.eval()

In [4]:
image_folder_path = './data/archive/images/images'
x_test_file_path = './data/x_test_final.csv'
y_data_file_path = './data/y_full.csv'

In [5]:
y_labels = pd.read_csv(y_data_file_path).columns.tolist()

In [6]:
test_dataset = CustomInferenceDataset(label_encoders=label_encoders, onehot_encoders=one_hot_encoders, x_test_path=x_test_file_path, image_folder_path=image_folder_path)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Inicializa una lista para almacenar los resultados
results = []

for i, (image, tabular_data, attribute_name, test_id) in enumerate(test_loader):
	image = image.to(device)
	tabular_data = tabular_data.to(device)

	test_id = test_id[0]
	attribute_name = attribute_name[0]

	with torch.no_grad():
		try:
			pred = model(image, tabular_data)
			print(f"Prediction {i}/{len(test_loader)}")

		except Exception as e:
			print(f'Error predicting for test_id {test_id}. Error message: {e}')
			results.append({'test_id': test_id, 'des_value': 'INVALID'})
			continue

	# Convertir predicciones a índices de clases
	pred_classes = [torch.argmax(attr_pred, dim=1).cpu().numpy() for attr_pred in pred]

	# Obtener el índice del atributo
	attr_index = y_labels.index(attribute_name)

	# Crear un vector one-hot para el índice predicho
	num_categories = len(test_dataset.onehot_encoders[attribute_name].categories_[0])
	onehot_vector = [0] * num_categories
	onehot_vector[pred_classes[attr_index][0]] = 1
	
	# Decodifica el valor de la clase predicha
	pred_value = test_dataset.value_from_onehot_encoder(attribute_name, onehot_vector)

	# Agrega el resultado a la lista
	results.append({'test_id': test_id, 'des_value': pred_value})

# Convierte la lista de resultados en un DataFrame
resulting_df = pd.DataFrame(results)

# Guarda el DataFrame en un archivo CSV
resulting_df.to_csv('./data/test_predictions.csv', index=False)
